#  Linear Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

Link to the dataset.
https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

A version of the dataset is in the ZIP file where you got this notebook.

In [ ]:
#put here your ``numero di matricola''
ID_number = 1 # COMPLETE

In [ ]:
# to get in-line plots
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats

## Import Data
Load the data from a .csv file

**TO DO: insert your ID number (matricola)**


In [ ]:
np.random.seed(ID_number)

filename = "kc_house_data.csv"

#load the data
df = pd.read_csv(filename, sep = ',')

#let's print out the data
print(df)

# A quick overview of data

Now let's clean the data and inspect it using the method describe().

In [ ]:
#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()
#for more interesting visualization: use Pandas!

Extract input and output data. We want to predict the price by using features other than id as input.

# Split data in training and test set

Given $m$ total data, keep $m_t$ data as training data, and $m_{test}:=m - m_t$ for test data. For instance one can take $m_t=\frac{3}{4}m $ of the data as training, and $m_{test}=\frac{m}{4}$ as testing. Let us define
- $S_{t}$ the training data set
- $S_{test}$ the testing data set


The reason for this splitting is as follows:

TRAINING DATA: The training data are used to compute the empirical loss
$$
L_S(h) = \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
which is used to get $h_S$ in a given model class ${\cal H}$.
i.e. 
$$
h_S = {\rm arg\; min}_{h \in {\cal H}} \, L_S(h)
$$

TESTING DATA: Last, the test data set can be used to estimate the performance of the chosen hypothesis $h_{S}$ using:

$$
L_{\cal D}(h_S)  \simeq \frac{1}{m_{test}} \sum_{ z_i \in S_{test}} \ell(h_{S},z_i)
$$

**TO DO: split the data in training and test sets (suggestion: use $m_t=\left\lfloor\frac{3}{4}m\right\rfloor $, $m_{test} = m-m_t$)**

In [ ]:
#let's consider only the values in the DataFrame
Data = df.values

# m = number of input samples
m = Data.shape[0]

print("Total number of samples: ", m)

#size of training dataset
size_training = # COMPLETE

print("Number of samples in training data: ", size_training)

#shuffle the data (to make sure we get a random split)

# COMPLETE

#divide data into matrix X of features and target vector Y 

Y = # COMPLETE
X = # COMPLETE

#training data

X_training = # COMPLETE
Y_training = # COMPLETE
print("Training input data size: ", X_training.shape)
print("Training output data size: ", Y_training.shape)

#test data, to be used to estimate the true loss of the final model
X_test = # COMPLETE
Y_test = # COMPLETE
print("Test input data size: ", X_test.shape)
print("Test output data size: ", Y_test.shape)

# Data Normalization

It is common practice in Statistics and Machine Learning to scale the data (= each variable) so that it is centered (zero mean) and has standard deviation equal to $1$. This helps in terms of numerical conditioning of the (inverse) problems of learning the model (the coefficients of the linear regression in this case), as well as to give the same scale to all the coefficients. 

In [ ]:
# scale the data: standardize the training feature matrix
from sklearn import preprocessing
scaler = # COMPLETE
X_training_scaled = # COMPLETE
print("Mean of the training input data:", X_training_scaled.mean(axis=0))
print("Std of the training input data:",X_training_scaled.std(axis=0))

# now we scale the test feature matrix using the same transformation used
# for the training dataset, since the weights of the model will be learned
# data scaled according to such transformation

X_test_scaled = # COMPLETE
print("Mean of the test input data:", X_test_scaled.mean(axis=0))
print("Std of the test input data:", X_test_scaled.std(axis=0))



# Model Training 

The model is trained minimizing the empirical error
$$
L_S(h) := \frac{1}{N_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
When the loss function is the quadratic loss
$$
\ell(h,z) := (y - h(x))^2
$$
we define  the Residual Sum of Squares (RSS) as
$$
RSS(h):= \sum_{z_i \in S_{t}} \ell(h,z_i) = \sum_{z_i \in S_{t}} (y_i - h(x_i))^2
$$ so that the training error becomes
$$
L_S(h) = \frac{RSS(h)}{m_t}
$$

We recal that, for linear models we have $h(x) = <w,x>$ and the Empirical error $L_S(h)$ can be written
in terms of the vector of parameters $w$ in the form
$$
L_S(w) = \frac{1}{m_t} \|Y - X w\|^2
$$
where $Y$ and $X$ are the matrices whose $i-$th row are, respectively, the output data $y_i$ and the input vectors $x_i^\top$.

The least squares solution is given by the expression
$$
\hat w = {\rm arg\;min}_w L_S(w) = (X^\top X)^{-1} X^\top Y
$$
When the matrix $X$ is not invertible (or even when it is invertible), the solution can be computed using the Moore-Penrose pseudonverse $(X^\top X)^{\dagger}$ of $(X^\top X)$
$$
\hat w = (X^\top X)^{\dagger} X^\top Y
$$
The Moore-Penrose pseudoinverse $A^\dagger$ of a matrix $A \in \mathbb{R}^{m\times n}$ can be expressed in terms of the Singular Value Decomposition (SVD) as follows:
let $A\in \mathbb{R}^{m\times n}$ be of rank $r\leq {\rm min}(n,m)$ and let  
$$
 A = USV^\top
 $$
 be the singular value decomposition of  $A$ where  
 $$
 S = {\rm diag}\{s_1,s_2,..,s_r\}
 $$
 Then 
 $$
 A^\dagger =V S^{-1} U^\top 
 $$
 
 In practice some of the singular values may be very small (e.g. $<1e-10$). Therefore it makes sense to  first approximate the matrix $A$ truncating the SVD and then using the pseudoinverse formula.
 
 More specifically, let us postulate that, given a threshold $T_h$ (e.g $=1e-12$), we have $\sigma_i<T_h$, for $i=\hat r + 1,..,r$. Then we can approximate (by SVD truncation) $A$ using:
 
 $$A = USV^\top =U \,{\rm diag}\{s_1,s_2,..,s_r\}\, V^\top \simeq \hat A_r = U\,{\rm diag}\{s_1,s_2,..,s_{\hat r}, 0,..,0\}\,V^\top
 $$
 So that 
 $$
 A^\dagger \simeq \hat A_r^\dagger:= V \,{\rm diag}\{1/s_1,1/s_2,..,1/s_{\hat r}, 0,..,0\}\, U^\top
 $$
  
 In numpy, the Moore-Penrose pseudo-inverse of a matrix can be computed using the method numpy.linalg.pinv(...), which takes among its parameters the threshold for truncating the singular values to 0.
  
 **TO DO: compute the linear regression coefficients according to the description above (using numpy.linalg.pinv(...) )**

In [ ]:
#compute linear regression coefficients for training data

#number of samples in the training set
m_training = X_training_scaled.shape[0]

#number of samples in the test set
m_test = X_test_scaled.shape[0]

# add a 1 at the beginning of each sample for training, and testing
# the numpy function hstack is useful for such operation
X_training_prime = # COMPLETE

X_test_prime = # COMPLETE

# set precision under which singular values are considered as zeros
prec = 1e-10  

# compute Moore-Penrose pseudoinverse of the matrix you need to compute 
# the weights of the model
A_inv = # COMPLETE

# now compute the weights and print them
w_hand = # COMPLETE

print("LS coefficients by hand:", w_hand)

# compute Residual Sums of Squares by hand
RSStr_hand = # COMPLETE

# print the RSS
print("RSS by hand:",  RSStr_hand)

# print the empirical risk
print("Empirical risk by hand:", RSStr_hand/m_training)

## Data prediction 

Compute the output predictions on both training and test set and compute the Residual Sum of Squares (RSS) defined above, the Empirical Loss and the quantity $R^2$ where
$$
R^2 = 1 - \frac{\sum_{z_i \in S_t} (\hat y_i - \bar y)^2}{\sum_{z_i \in S_t} (y_i - \bar y)^2} \quad \quad \bar y = \frac{1}{m_t} \sum_{z_i \in S_t} y_i
$$
is the so-called "Coefficient of determination" (COD).

**TO DO Compute these quantities on  training and test data.**


In [ ]:
#compute predictions on training and test
prediction_training = # COMPLETE
prediction_test = # COMPLETE

#what about the RSS and empirical loss for points in the test data?
RSS_test = # COMPLETE

print("RSS on test data:",  RSS_test)
print("Generalization error estimated on test data (i.e., empirical loss on test data):", RSS_test/m_test)

#another measure of how good our linear fit is given by the following (that is R^2)
measure_training = # COMPLETE
measure_test = # COMPLETE

print("Measure on Training Data (R^2):", measure_training)
print("Measure on Test Data(R^2):", measure_test)

## ... and plot:


### (1) output predictions on training  data

In [ ]:
# Plot predictions on Training data 
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_training[0:m_training])), key=lambda k: prediction_training[0:m_training][k])
plt.plot(Y_training[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_training[sorting_permutation], 'rx')

plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Training Data')
plt.show()

### (2) output predictions on test  data

In [ ]:
# Plot predictions on test data 
plt.figure()

# COMPLETE

## Ordinary Least-Squares using scikit-learn

A fast way to compute the LS estimate is through sklearn.linear_model

In [ ]:
# Remove the ``ones'' column in the features matrix (sklearn inserts it automatically)

In [ ]:
X_training_OLS = X_training_scaled[:,1:]
X_test_OLS = X_test_scaled[:,1:]

In [ ]:
from sklearn import linear_model
LinReg = linear_model.LinearRegression()  # build the object LinearRegression
LinReg.fit(X_training_OLS, Y_training)  # estimate the LS coefficients
print("Intercept:", LinReg.intercept_)
print("Least-Squares Coefficients:", LinReg.coef_)
prediction_training = LinReg.predict(X_training_OLS)  # predict output values on training set
prediction_test = LinReg.predict(X_test_OLS)  # predict output values on test set
print("Measure on training data:", LinReg.score(X_training_OLS, Y_training))
print("Measure on test data:", LinReg.score(X_test_OLS, Y_test))